## Multi-label Legal Text Classification for CIA
(*Paragraph parsing*)

In [1]:
# main
import pandas as pd
import numpy as np

In [2]:
# parsing
from glob import glob
import os 
import re

import pdfplumber
from pdfminer.high_level import extract_text
from langdetect import detect, DetectorFactory

In [3]:
os.chdir("../..")
os.getcwd()

'/Users/janinedevera/Documents/School/MDS 2021-2023/Thesis/multilabel-legal-text-classification-CIA'

### A. Legal Texts

List of paths for PDFs

In [211]:
countries = ["Philippines", "Singapore"]
dirs = []
for country in countries:
    dir = "data/legislation/" + str(country) + "/"
    dirs.append(dir)

pdf_list = []
for path in dirs:
    list = glob(os.path.join(path, "*.pdf"))
    pdf_list.extend(list)

len(pdf_list)

36

In [209]:
pdf_list

['data/legislation/Philippines/PH_RA 9337 Amendments to Internal Revenue Code 2005.pdf',
 'data/legislation/Philippines/PH_RA 7354 Postal Service Act 1992.pdf',
 'data/legislation/Philippines/PH_EO 292 Administrative Code 1987.pdf',
 'data/legislation/Philippines/PH_RA 7160 Local Government Code 1991.pdf',
 'data/legislation/Philippines/PH_RA 11232 Revised Corporation Code 2019.pdf',
 'data/legislation/Philippines/PH_CA 146 Public Service Act 1936.pdf',
 'data/legislation/Philippines/PH_RA 4136 Land Transportation Act 1964.pdf',
 'data/legislation/Philippines/PH_EO 24 2011.pdf',
 'data/legislation/Philippines/PH_RA 9184 Government Procurement Act IRR 2016.pdf',
 'data/legislation/Philippines/PH_EO 203.pdf',
 'data/legislation/Philippines/PH_RA 11032 EODB 2017.pdf',
 'data/legislation/Philippines/PH_EO 80 2012.pdf',
 'data/legislation/Philippines/PH_RA 8794 Motor Vehicle User Charge Law 2000.pdf',
 'data/legislation/Philippines/PH_EO 7 2010.pdf',
 'data/legislation/Philippines/PH_EO 47 

Parse PDFs with pdfplumber

In [212]:
def text_split(text):
    # find double line breaks
    if text.find("\n\n") != -1:
        split = text.split("\n\n")
    elif text.find("\n \n") != -1:
        split = text.split("\n \n")
    # find section/article divisions
    elif len(re.findall('\\n[A-Z]{2,}', text)) > 0:
        split = re.split('\\n[A-Z]{2,}', text)
    elif len(re.findall('\\nSection', text)) > 0:
        split = re.split('\\nSection', text)
    # find single line breaks
    elif text.find('\n“') != -1:
        split = text.split('\n“')
    elif text.find('\n') != -1:
        split = text.split('\n')
    else: 
        split = text
    return split
        

In [166]:
def text_split(text):
    # find double line breaks
    if text.find("\n\n") != -1:
        split = text.split("\n\n")
    elif text.find("\n \n") != -1:
        split = text.split("\n \n")
        if len(split) < 5:
            for line in split:
                split = []
                # find section/article divisions
                if len(re.findall('\\n[A-Z]{2,}', line)) > 0:
                    split2 = re.split('\\n[A-Z]{2,}', line)
                elif len(re.findall('\\nSection', line)) > 0:
                    split2 = re.split('\\nSection', line)
                # find single line breaks
                elif text.find('\n“') != -1:
                    split2 = text.split('\n“')
                elif text.find('\n') != -1:
                    split2 = text.split('\n')
                split.append(split2)
        else: 
            split = split
    else: 
        split = text
    return split
        

Parsing by document

In [98]:
## DO NOT RUN

df_text = pd.DataFrame(columns = ['file_name', 'text', 'lang'])

for pdf_file in pdf_list:
        # search for pdf file name from path 
        file_name = re.search('[a-zA-Z0-9]*_([a-zA-Z0-9]+)[^\\>.]+', pdf_file)[0]
        with pdfplumber.open(pdf_file) as pdf:
            text = []
            # load pdf by page 
            for i in range(len(pdf.pages)):
                page = pdf.pages[i]
                page_text = page.extract_text()
                text.append(page_text)
            text = ' '.join(text)
            lang = detect(text)
        row = pd.DataFrame({'file_name': file_name,
                                'text': text, 'lang': lang}, index=[0])
        df_text = pd.concat([row,df_text.loc[:]]).reset_index(drop=True)

Parsing by paragraph

In [ ]:
df_text = pd.DataFrame(columns = ['file_name', 'text', 'lang'])

for pdf_file in pdf_list:
        # search for pdf file name from path 
        file_name = re.search('[a-zA-Z0-9]*_([a-zA-Z0-9]+)[^\\>.]+', pdf_file)[0]
        with pdfplumber.open(pdf_file) as pdf:
            text = []
            # load pdf by page 
            for i in range(len(pdf.pages)):
                page = pdf.pages[i]
                page_text = page.extract_text()
                text.append(page_text)
            text = ' '.join(text)
            lang = detect(text)
            # split by paragraph 
            paragraphs = text_split(text)
            for p in paragraphs:
                paragraph = p
                # create dataframe
                row = pd.DataFrame({'file_name': file_name,
                        'text': paragraph, 'lang': lang}, index=[0])
                df_text = pd.concat([row,df_text.loc[:]]).reset_index(drop=True)

In [214]:
df_text

,file_name,text,lang
0,SG_Public Consultation Paper on Draft Amendmen...,Page 14 of 14,en
1,SG_Public Consultation Paper on Draft Amendmen...,"52. For the avoidance of doubt, all the infor...",en
2,SG_Public Consultation Paper on Draft Amendmen...,51. MCI/IMDA reserves the right to make ...,en
3,SG_Public Consultation Paper on Draft Amendmen...,50. All submissions should reach MCI/IMDA wit...,en
4,SG_Public Consultation Paper on Draft Amendmen...,Supporting materials may be placed as an annex...,en
...,...,...,...
7149,PH_RA 9337 Amendments to Internal Revenue Code...,1. Section 27 of the National Internal Revenu...,en
7150,PH_RA 9337 Amendments to Internal Revenue Code...,"AMENDED, AND FOR OTHER PURPOSES\nBe it enacte...",en
7151,PH_RA 9337 Amendments to Internal Revenue Code...,"ACT AMENDING SECTIONS 27, 28, 34, 106, 107, 1...",en
7152,PH_RA 9337 Amendments to Internal Revenue Code...,"(/)\nRepublic Act No. 9337\nSigned on May 24,...",en


In [215]:
df_text.file_name.value_counts()

PH_RA 9184 Government Procurement Act IRR 2016          1608
SG_Postal Competition Code 2017                          964
PH_RA 11232 Revised Corporation Code 2019                703
PH_RA 7160 Local Government Code 1991                    680
PH_Constitution Republic of the Philippines 1987         355
PH_RA 9184 Government Procurement Act 2002               273
PH_EO 292 Administrative Code 1987                       240
SG_License to Provide Postal Services Asendia 2017       232
SG_License to Provide Postal Services SP 2017            224
SG_Guidelines for Postal Service Operations 2019         217
SG_License to Provide Postal Services DHL 2019           213
SG_License to Provide Postal Services WMG 2018           206
SG_Postal Accounting Separation Framework 2010           169
SG_Postal Services Operations Code 2017                  160
PH_RA 7354 Postal Service Act 1992                       151
SG_Public Consultation Paper on Draft Amendment 2020     115
PH_RA 4136 Land Transpor

In [222]:
sample = df_text[df_text['file_name'] == "PH_RA 11232 Revised Corporation Code 2019"]
sample

,file_name,text,lang
5341,PH_RA 11232 Revised Corporation Code 2019,Published in Manila Bulletin and Business Mirr...,en
5342,PH_RA 11232 Revised Corporation Code 2019,,en
5343,PH_RA 11232 Revised Corporation Code 2019,,en
5344,PH_RA 11232 Revised Corporation Code 2019,,en
5345,PH_RA 11232 Revised Corporation Code 2019,,en
...,...,...,...
6039,PH_RA 11232 Revised Corporation Code 2019,Be it enacted by the Senate and House of Repre...,en
6040,PH_RA 11232 Revised Corporation Code 2019,AN ACT PROVIDING FOR THE REVISED CORPORATION \...,en
6041,PH_RA 11232 Revised Corporation Code 2019,______________ \n [ REPUBLIC ACT NO. 11232 ],en
6042,PH_RA 11232 Revised Corporation Code 2019,"\nBegun and held in Metro Manila, on Monday, ...",en
